=================================================
Milestone Phase 1

Nama  : Kamelia Ramandha
Batch : CODA-RMT-010

Program ini dibuat untuk melakukan webscrapping kepada website nordstorm dan melakukan analisa sederhana terhadap data
=================================================

In [273]:
# Mengimport library yang dibutuhkan

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver


# Membuat list kosong
brand = []
product_name = []
price = []
var = []


# Scrapping data 5 halaman pada website Nordstorm
for page in range(1,6):
    weblink = f'https://www.nordstrom.com/browse/women/handbags?offset=11&page={page}'
    driver = webdriver.Chrome()
    driver.get (weblink)
    time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

# Mengambil beberapa data dari webpage
    for kotak in soup.find_all('article', {'class': 'U5O1_ _90bt zzWfq RpUx3'}):
        try:
            # Setelah ditelaah, brand memiliki tags dan attribute value yang berbeda, maka dibuat 2 kemungkinan
            brand_div = kotak.find('div', {'class': 'KtWqU jgLpg Y9bA4 Io521'})
            brand_strong = kotak.find('h3', {'class': 'kKGYj Y9bA4'}).find('strong') if kotak.find('h3', {'class': 'kKGYj Y9bA4'}) else None

            if brand_div:
                brand.append(brand_div.get_text())
            else:
                brand.append(brand_strong.get_text())
        except:
            brand.append(None)
        
        # Mengambil product name
        try:
            product_name.append(kotak.find('a', {'class': 'dls-ogz194'}).get_text())
        except:
            product_name.append(None)
        
        # Mengambil harga
        try:
            price.append(kotak.find('span', {'class': 'qHz0a EhCiu dls-ihm460'}).get_text())
        except:
            price.append(None)
        
        # Mengambil variasi product yang ada di dalam button
        try:
            buttons = kotak.find_all('button', {'class': 'D2TsF'})
            labels = set()
            for button in buttons:
                label = button.get('aria-label')
                if label:
                    labels.add(label)
            var.append(labels if labels else None)   # set, bukan list
        except:
            var.append(None)
        


python(66995) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67032) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67069) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67106) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67176) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67215) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67270) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67307) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67357) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67396) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [274]:
# Menutup driver setelah scrapping data

driver.close()

In [275]:
# Memasukan data yang sudah di ambil ke dalam dataframe
df = pd.DataFrame({
    'brand': brand,
    'product_name': product_name,
    'price': price,
    'var': var
})

print(df)


                    brand                                       product_name  \
0    Donna Karan New York            Donna Karan New York Amagansett Satchel   
1              Tory Burch                      Tory Burch Mini Ella Bio Tote   
2              Jimmy Choo           Jimmy Choo Bon Bon Drawstring Mesh Pouch   
3              rag & bone             rag & bone Gramercy Medium Satchel Bag   
4                   COACH  COACH Willow Polished Pebble Leather Shoulder Bag   
..                    ...                                                ...   
133   Christian Louboutin     Christian Louboutin Loubi54 Flat Leather Pouch   
134           Marc Jacobs  Marc Jacobs The Dual Mini Crescent Leather Sho...   
135   Herschel Supply Co.  Herschel Supply Co. Novel Duffle Bag (Nordstro...   
136            MZ Wallace  MZ Wallace Small Astor Convertible Quilted Lea...   
137         Saint Laurent  Saint Laurent Medium Loulou Matelassé Leather ...   

      price                            

In [276]:
# Menyimpan data ke csv

df.to_csv('nordstorm_bag.csv', index = False)

In [277]:
# Melihat tipe dan jumlah data dari setiap set
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         138 non-null    object
 1   product_name  138 non-null    object
 2   price         107 non-null    object
 3   var           91 non-null     object
dtypes: object(4)
memory usage: 4.4+ KB


In [278]:
# Membersihkan data price
df["starting_price"] = (
    df["price"]
    .str.split("–")                     # pisahkan berdasarkan tanda "-"
    .str[0]                             # ambil bagian pertama sebelum "-"
    .str.strip()                        # hapus spasi di kiri/kanan
    .str.replace("$", "", regex=False)  # hapus simbol $
    .str.replace(",", "", regex=False)  # hapus koma ribuan
    .astype(float)                      # ubah ke float
)


In [279]:
# Melihat info yang baru setelah diedit
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           138 non-null    object 
 1   product_name    138 non-null    object 
 2   price           107 non-null    object 
 3   var             91 non-null     object 
 4   starting_price  107 non-null    float64
dtypes: float64(1), object(4)
memory usage: 5.5+ KB


In [280]:
# Menghapus column price yang sudah tidak terpakai
df = df.drop(columns=["price"])

In [281]:
# Membersihkan data None agar lebih bersih ketika diolah
df = df.dropna()


In [282]:
# Melihat data terakhir setelah dibersihkan
df

,brand,product_name,var,starting_price
1,Tory Burch,Tory Burch Mini Ella Bio Tote,"{clay, gray, light sand, black, warm white}",260.0
3,rag & bone,rag & bone Gramercy Medium Satchel Bag,"{chocolate, black}",458.0
4,COACH,COACH Willow Polished Pebble Leather Shoulder Bag,"{black, chalk}",395.0
6,NAGHEDI,NAGHEDI Nomad Hobo,"{ecru, celadon, cashmere, onyx, chocolate, ink...",315.0
7,COACH,COACH Woven Leather Swing Tote,"{black, maple}",495.0
...,...,...,...,...
126,Olga Berg,Olga Berg Everly Handwoven Faux Leather Frame ...,"{white, silver, natural, gold, tangerine, black}",139.0
127,Mansur Gavriel,Mansur Gavriel Candy Baguette Leather Shoulder...,"{warm grey, chocolate, black, oxblood}",595.0
128,Kate Spade New York,Kate Spade New York large deco pebble leather ...,"{blackberry jam, timeless taupe, black, burnt ...",478.0
129,Clare V.,Clare V. Petit Moyen Woven Messenger Bag,"{natural woven checker, fern mist woven checke...",385.0


In [283]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66 entries, 1 to 130
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           66 non-null     object 
 1   product_name    66 non-null     object 
 2   var             66 non-null     object 
 3   starting_price  66 non-null     float64
dtypes: float64(1), object(3)
memory usage: 2.6+ KB


In [284]:
# Menyimpan data bersih ke csv
df.to_csv('nordstorm_bag_clean.csv', index = False)